In [118]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string

In [181]:
file_path='/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/electives.xlsx'

In [182]:
sheet_names = pd.ExcelFile(file_path).sheet_names


In [183]:
CD = pd.read_excel(file_path, sheet_name='CD')
AI = pd.read_excel(file_path, sheet_name='AI')
ML = pd.read_excel(file_path, sheet_name='ML')
BDA = pd.read_excel(file_path, sheet_name='BDA')
IS = pd.read_excel(file_path, sheet_name='IS')
MCWC = pd.read_excel(file_path, sheet_name='MCWC')
CC = pd.read_excel(file_path, sheet_name='CC')
MAD = pd.read_excel(file_path, sheet_name='MAD')
DF = pd.read_excel(file_path, sheet_name='DF')

In [186]:
def update_student_subjects(df, subject_name,student_subjects):
    for _, row in df.iterrows():
        student_key = (row['enrollment'], row['name'])  # Create a tuple of (enrollment, name)
        if student_key in student_subjects:
            student_subjects[student_key].append(subject_name)
        else:
            student_subjects[student_key] = [subject_name]


In [187]:
student_subjects = {}
update_student_subjects(CD,'CD',student_subjects)
update_student_subjects(AI,'AI',student_subjects)
update_student_subjects(ML,'ML',student_subjects)
update_student_subjects(BDA,'BDA',student_subjects)
update_student_subjects(IS,'IS',student_subjects)
update_student_subjects(MCWC,'MCWC',student_subjects)
update_student_subjects(CC,'CC',student_subjects)
update_student_subjects(MAD,'MAD',student_subjects)
update_student_subjects(DF,'DF',student_subjects)

In [188]:
min_div_size =40
max_div_size = 66
def calculate_divisions(student_count):    
    min_divisions = (student_count + max_div_size - 1) // max_div_size
    max_divisions = student_count // min_div_size        
    for divisions in range(min_divisions, max_divisions + 1):
        div_size = student_count // divisions
        if min_div_size <= div_size <= max_div_size:
            return divisions        
    return min_divisions

In [189]:
total_hours = {
    "IOT":4,
    "DV":4,
    "WP":5,
    "DM":5,
    "AJAVA":5,
    "SS":5,
}
total_student_count = len(student_subjects.keys())
approx_division_count = calculate_divisions(total_student_count)
approx_division_count
# the total teaching our for each subject will be in multiplication of the approx division count

# for subject in total_hours:
#     total_hours[subject] *= approx_division_count
# total_hours

3

In [190]:
subject_groups = {}
for student, subject_group in student_subjects.items():
    # Convert subject_group to a tuple if it isn't one already
    subject_group_tuple = tuple(subject_group)
    if subject_group_tuple not in subject_groups:
        subject_groups[subject_group_tuple] = [student]
    else:
        subject_groups[subject_group_tuple].append(student)

In [191]:
for subject_group in subject_groups:
    print(f"Group - {subject_group} | Student Count {len(subject_groups[subject_group])}")

Group - ('CD', 'AI', 'ML', 'IS', 'CC', 'DF') | Student Count 138
Group - ('CD', 'AI', 'BDA', 'IS', 'MCWC', 'MAD') | Student Count 9
Group - ('CD', 'AI', 'BDA', 'IS', 'CC', 'DF') | Student Count 9
Group - ('CD', 'AI', 'ML', 'IS', 'CC', 'MAD') | Student Count 6
Group - ('CD', 'AI', 'BDA', 'IS', 'CC', 'MAD') | Student Count 1
Group - ('CD', 'AI', 'BDA', 'IS', 'MCWC', 'DF') | Student Count 10
Group - ('CD', 'AI', 'ML', 'IS', 'MCWC', 'MAD') | Student Count 2
Group - ('CD', 'AI', 'ML', 'IS', 'MCWC', 'DF') | Student Count 3


In [127]:
BDA,MAD,MCWC
ML,IS,CC

(      no    enrollment                               name
 0      1  200280107145  Prajapati Jay Kumar Dashrathbhai 
 1      2  210280107001          Ratadiya Mayur Devsibhai 
 2      3  210280107003                    Taral Bambhania
 3      4  210280107004           Darshil Nareshbhai Patel
 4      5  210280107005                     Akshar Chauhan
 ..   ...           ...                                ...
 144  145  220283107035     SIDDHAPURA TRUSHTI MAHESHBHAI 
 145  146  220283107036              SINGH PREM SHIVPOOJAN
 146  147  220283107039                    Himani Trivedi 
 147  148  220283107041           Vara Shruti Jagdishbhai 
 148  149  220283107042                         Aryan wala
 
 [149 rows x 3 columns],
       no    enrollment                               name
 0      1  200280107145  Prajapati Jay Kumar Dashrathbhai 
 1      2  210280107001          Ratadiya Mayur Devsibhai 
 2      3  210280107003                    Taral Bambhania
 3      4  210280107004      

In [128]:
divisions = {}
for i in range(approx_division_count):
    divisions[string.ascii_uppercase[i]] = []
divisions

{'A': [], 'B': []}

In [129]:
remaining_students_count = total_student_count
for division in divisions:        
    # check the count of students in current division
    students_count_in_div = len(divisions[division])
    capacity_remained_in_div = max_div_size-students_count_in_div
    if capacity_remained_in_div >= remaining_students_count:
        # insert all the remaining students in current div
        for subject_group,students in subject_groups.items():
            divisions[division] += students
    else:
        # iterate through the subject groups
        # check if any available subject group has the student count greater than the max_div_count
        subject_group_gt_max_div_count= list(filter(lambda subject_group: len(subject_groups[subject_group]) > max_div_size,subject_groups))
        if subject_group_gt_max_div_count:
            # move the first max_div_count students to this division
            divisions[division] = subject_groups[subject_group_gt_max_div_count[0]][:max_div_size]
            subject_groups[subject_group_gt_max_div_count[0]] = subject_groups[subject_group_gt_max_div_count[0]][max_div_size:]
            # also reduce the remaining student count            
            remaining_students_count -= max_div_size
            # now this div's full so move let's move to next div            
            continue
            

In [192]:
# Now to sort the students according to their enrollment in each divisions
for division in divisions:
    divisions[division]=sorted(divisions[division], key=lambda student: student[0])
divisions

{'A': [(200280107145, 'Prajapati Jay Kumar Dashrathbhai '),
  (210280107001, 'Ratadiya Mayur Devsibhai '),
  (210280107003, 'Taral Bambhania'),
  (210280107004, 'Darshil Nareshbhai Patel'),
  (210280107005, 'Akshar Chauhan'),
  (210280107006, 'Patel Nisarg Surendrakumar'),
  (210280107007, 'DOBARIYA HAPPY RAMNIKBHAI '),
  (210280107008, 'Meet Soni '),
  (210280107009, 'Nirali Darji'),
  (210280107010, 'Devmurari Maulik Vijaybhai'),
  (210280107011, 'Justin George'),
  (210280107012, 'Patel Dhruvit Vasantbhai '),
  (210280107013, 'Halpati Divya DipakKumar '),
  (210280107014, 'Jiyani Hasti Mukeshbhai'),
  (210280107015, 'Astha Hareshbhai Viradiya '),
  (210280107016, 'Kaman Devanshi KaushikKumar '),
  (210280107017, 'Naik Chinmay Dharmesh'),
  (210280107018, 'Patel Chirag Sunilbhai'),
  (210280107019, 'PRAJAPATI HARSH DHIRENDRAKUMAR '),
  (210280107020, 'KERALIYA KARN NAROTTAMBHAI'),
  (210280107022, 'Henish Kantibhai Naliyadhara '),
  (210280107024, 'Jha Kunjbihari Vinodanand '),
  (21

In [195]:
subject_groups = {
('AI','CD','ML', 'IS', 'CC', 'DF'):138,
('AI','CD','ML', 'IS', 'CC', 'MAD'):6,
('AI','CD','ML', 'IS', 'MCWC', 'MAD'):2,
('AI','CD','ML', 'IS', 'MCWC', 'DF'):3,
('AI','CD','BDA', 'IS', 'MCWC', 'MAD'):9,
('AI','CD','BDA', 'IS', 'CC', 'DF'):9,
('AI','CD','BDA', 'IS', 'CC', 'MAD'):1,
('AI','CD','BDA', 'IS', 'MCWC', 'DF'):10,
}

total_hours = {
    "ML":5,
    "BDA":5,
    "IS":5,
    "MCWC":3,
    "CC":3,
    "MAD":4,
    "DF":4,
    "CD":5,
    "AI":5
}
# Constraints
min_students = 0  # No minimum size for divisions
max_students = 90  # Maximum size per division

def allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students):
    # Step 1: Preprocess subject groups to split any group exceeding max_students
    preprocessed_groups = []
    for group, count in subject_groups.items():
        while count > max_students:            
            preprocessed_groups.append((group, max_students))
            count -= max_students
        if count > 0:
            preprocessed_groups.append((group, count))        
    # Step 2: Initialize variables for the backtracking algorithm
    groups = [group for group, _ in preprocessed_groups]
    students = [count for _, count in preprocessed_groups]    
    best_solution = {"divisions": None, "deviation": float("inf")}
    recursive_calls = 0

    def calculate_hours(division):
        """Calculate total hours per subject in a division."""
        hours = {subject: 0 for subject in total_hours}
        for group, _ in division:
            for subject in group:
                hours[subject] = total_hours[subject]
        return hours

    def calculate_deviation(hours):
        """Calculate the total deviation of hours from the initial requirement."""
        return sum(abs(hours[sub] - total_hours[sub]) for sub in total_hours)

    def backtrack(current_divisions, remaining_groups, remaining_counts):
        nonlocal recursive_calls
        recursive_calls += 1

        # Base case: no groups left to allocate
        if not remaining_groups:            
            total_hours_all = {subject: 0 for subject in total_hours}
            for division in current_divisions:
                hours = calculate_hours(division)
                for subject in total_hours_all:
                    total_hours_all[subject] += hours[subject]
            deviation = calculate_deviation(total_hours_all)
            if deviation < best_solution["deviation"]:
                best_solution["divisions"] = [div[:] for div in current_divisions]
                best_solution["deviation"] = deviation
            return

        # Get the next group and its student count
        group = remaining_groups[0]
        count = remaining_counts[0]

        # Try allocating the group (or part of it) to each division
        for i in range(len(current_divisions)):
            division = current_divisions[i]
            division_student_count = sum(student_count for _, student_count in division)

            if division_student_count + count <= max_students:
                # Allocate the entire group to this division
                division.append((group, count))
                backtrack(
                    current_divisions,
                    remaining_groups[1:],
                    remaining_counts[1:],
                )
                # Backtrack
                division.pop()
            else:
                # Allocate a part of the group to this division
                split_count = max_students - division_student_count
                if split_count > 0:
                    # Add the split part to this division
                    division.append((group, split_count))
                    # Recurse with the remaining students of the group
                    backtrack(
                        current_divisions,
                        [group] + remaining_groups[1:],
                        [count - split_count] + remaining_counts[1:],
                    )
                    # Backtrack
                    division.pop()

        # Start a new division with this group
        if len(current_divisions) < len(groups):
            current_divisions.append([(group, count)])
            backtrack(
                current_divisions,
                remaining_groups[1:],
                remaining_counts[1:],
            )
            # Backtrack
            current_divisions.pop()

    # Step 3: Initialize backtracking
    backtrack([], groups, students)

    # Log statistics
    print(f"Total Recursive Calls: {recursive_calls}")

    # Calculate the final subject hours for each subject
    total_hours_all_divisions = {subject: 0 for subject in total_hours}
    for division in best_solution["divisions"]:
        division_hours = calculate_hours(division)
        for subject in total_hours_all_divisions:
            total_hours_all_divisions[subject] += division_hours[subject]

    # Return the best solution with subject hours included
    return best_solution, total_hours_all_divisions


# Run the revised algorithm
result, subject_hours = allocate_groups_with_splitting(subject_groups, total_hours, min_students, max_students)
print("\nOptimal Division Allocation:")
if result["divisions"]:
    for i, division in enumerate(result["divisions"], start=1):        
        division_students = sum(count for _, count in division)
        print(f"Division {i}: {division}, Total Students: {division_students}")
        
# Print total deviation
print("\nTotal Deviation:", result["deviation"])

# Print the subject hours for each subject in the final solution
print("\nFinal Subject Hours Across All Divisions:")
for subject, hours in subject_hours.items():
    print(f"{subject}: {hours} hours")

Total Recursive Calls: 5297

Optimal Division Allocation:
Division 1: [(('AI', 'CD', 'ML', 'IS', 'CC', 'DF'), 90)], Total Students: 90
Division 2: [(('AI', 'CD', 'ML', 'IS', 'CC', 'DF'), 48), (('AI', 'CD', 'ML', 'IS', 'CC', 'MAD'), 6), (('AI', 'CD', 'ML', 'IS', 'MCWC', 'MAD'), 2), (('AI', 'CD', 'ML', 'IS', 'MCWC', 'DF'), 3), (('AI', 'CD', 'BDA', 'IS', 'MCWC', 'MAD'), 9), (('AI', 'CD', 'BDA', 'IS', 'CC', 'DF'), 9), (('AI', 'CD', 'BDA', 'IS', 'CC', 'MAD'), 1), (('AI', 'CD', 'BDA', 'IS', 'MCWC', 'DF'), 10)], Total Students: 88

Total Deviation: 27

Final Subject Hours Across All Divisions:
ML: 10 hours
BDA: 5 hours
IS: 10 hours
MCWC: 3 hours
CC: 6 hours
MAD: 4 hours
DF: 8 hours
CD: 10 hours
AI: 10 hours
